# Алгоритм построения uplift-дерева

https://towardsdatascience.com/decision-tree-from-scratch-in-python-46e99dfea775
https://anderfernandez.com/en/blog/code-decision-tree-python-from-scratch/

Критерий разбиения: DeltaDeltaP

Оценка uplift в вершине дерева: разность средних значений целевой переменной Y в целевой и контрольной группах

In [1]:
import numpy as np
import pandas as pd
import time

### Delta-delta-p example

In [2]:
data = pd.read_excel('test_1k.xlsx')

In [3]:
data = data.sample(frac=1).reset_index(drop=True)

In [4]:
X = np.array(data['x'])
y = np.array(data['y'])
treatment = np.array(data['t'])

In [5]:
threshold = .3

In [6]:
control = [1-t for t in treatment]

In [7]:
left = [1 if x <= threshold else 0 for x in X]
right = [1 if l == 0 else 0 for l in left]

In [8]:
left_t = np.array(left) * np.array(treatment)
left_c = np.array(left) * np.array(control)

right_t = np.array(right) * np.array(treatment)
right_c = np.array(right) * np.array(control)

In [9]:
left_t_value = y * left_t
left_c_value = y * left_c
right_t_value = y * right_t
right_c_value = y * right_c

In [10]:
left_t_sum = sum(left_t_value)
left_c_sum = sum(left_c_value)
right_t_sum = sum(right_t_value)
right_c_sum = sum(right_c_value)

In [11]:
left_t_count = sum(left_t)
left_c_count = sum(left_c)
right_t_count = sum(right_t)
right_c_count = sum(right_c)

In [12]:
left_t_count + left_c_count + right_t_count + right_c_count

1000

In [13]:
uplift_left = (left_t_sum / left_t_count) - (left_c_sum / left_c_count)
uplift_right = (right_t_sum / right_t_count) - (right_c_sum / right_c_count)

In [14]:
uplift_left

0.007138491416129655

In [15]:
uplift_right

0.10650265594186581

In [16]:
delta_delta = np.abs(uplift_left - uplift_right)

In [17]:
delta_delta

0.09936416452573615

### Get thresholds list

In [18]:
X = np.load('example_X.npy')
treatment = np.load('example_treatment.npy')
y = np.load('example_y.npy')

example_preds = np.load('example_preds.npy')

In [19]:
n_features = X.shape[1]

### Find best thr

In [20]:
min_samples_leaf = 6000
min_samples_leaf_treated = 2500
min_samples_leaf_control = 2500

delta_delta_best = 0
threshold_best = 0
idx_best = 0

for idx in range(n_features):
    column_values = np.array(X[:, idx])

    unique_values = np.unique(column_values)
    if len(unique_values) > 10:
        percentiles = np.percentile(column_values, [3, 5, 10, 20, 30, 50, 70, 80, 90, 95, 97])
    else:
        percentiles = np.percentile(unique_values, [10, 50, 90])

    threshold_options = np.unique(percentiles)

    for threshold in threshold_options:
        control = [1-t for t in treatment]

        left = [1 if x <= threshold else 0 for x in column_values]
        right = [1 if l == 0 else 0 for l in left]

        left_t = np.array(left) * np.array(treatment)
        left_c = np.array(left) * np.array(control)

        right_t = np.array(right) * np.array(treatment)
        right_c = np.array(right) * np.array(control)

        left_t_value = y * left_t
        left_c_value = y * left_c
        right_t_value = y * right_t
        right_c_value = y * right_c

        left_t_sum = sum(left_t_value)
        left_c_sum = sum(left_c_value)
        right_t_sum = sum(right_t_value)
        right_c_sum = sum(right_c_value)

        left_t_count = sum(left_t)
        left_c_count = sum(left_c)
        right_t_count = sum(right_t)
        right_c_count = sum(right_c)

        uplift_left = (left_t_sum / left_t_count) - (left_c_sum / left_c_count)
        uplift_right = (right_t_sum / right_t_count) - (right_c_sum / right_c_count)

        delta_delta = np.abs(uplift_left - uplift_right)
        
        if delta_delta > delta_delta_best:
            if  left_t_count + left_c_count > min_samples_leaf and \
                right_t_count + right_c_count > min_samples_leaf and \
                left_t_count > min_samples_leaf_treated and \
                right_t_count > min_samples_leaf_treated and \
                left_c_count > min_samples_leaf_control and \
                right_c_count > min_samples_leaf_control:
                
                    delta_delta_best = delta_delta
                    threshold_best = threshold
                    idx_best = idx_best
                    print(idx_best, threshold_best, delta_delta_best)
    print('######')
print(idx_best, threshold_best, delta_delta_best)

0 -0.8343029365043767 1.7339620108477065
0 0.8428329389786856 1.7676766601226297
######
######
######
######
######
0 0.8428329389786856 1.7676766601226297


### Test

In [21]:
X = np.load('example_X.npy')
treatment = np.load('example_treatment.npy')
y = np.load('example_y.npy')

column_values = X[:, 0]
threshold =  0.8428329389786856

control = [1-t for t in treatment]

left = [1 if x <= threshold else 0 for x in column_values]
right = [1 if l == 0 else 0 for l in left]

left_t = np.array(left) * np.array(treatment)
left_c = np.array(left) * np.array(control)

right_t = np.array(right) * np.array(treatment)
right_c = np.array(right) * np.array(control)

left_t_value = [i * lt for i, lt in list(zip(y, left_t))]
left_c_value = [i * lc for i, lc in list(zip(y, left_c))]
right_t_value = [i * rt for i, rt in list(zip(y, right_t))]
right_c_value = [i * rc for i, rc in list(zip(y, right_c))]

left_t_sum = sum(left_t_value)
left_c_sum = sum(left_c_value)
right_t_sum = sum(right_t_value)
right_c_sum = sum(right_c_value)

left_t_count = sum(left_t)
left_c_count = sum(left_c)
right_t_count = sum(right_t)
right_c_count = sum(right_c)

uplift_left = (left_t_sum / left_t_count) - (left_c_sum / left_c_count)
uplift_right = (right_t_sum / right_t_count) - (right_c_sum / right_c_count)

delta_delta = np.abs(uplift_left - uplift_right)

print(threshold, delta_delta)

0.8428329389786856 1.7676766601226297


### Load data

In [22]:
# load data
X = np.load('example_X.npy')
treatment = np.load('example_treatment.npy')
y = np.load('example_y.npy')

### Model

In [23]:
import numpy as np

class Node:
    def __init__(self, predicted_class):
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None


class UpliftTreeRegressor:
    
    def __init__(
        self,
        max_depth: int = 3,
        min_samples_leaf: int = 1000,
        min_samples_leaf_treated: int = 300,
        min_samples_leaf_control: int = 300,
    ):
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.min_samples_leaf_treated = min_samples_leaf_treated
        self.min_samples_leaf_control = min_samples_leaf_control
        
    def fit(self, X, treatment, y):
        self.n_classes_ = len(set(treatment))
        self.n_features_ = X.shape[1]
        self.tree_ = self._grow_tree(X, y, treatment)

    def predict(self, X):
        return [self._predict(inputs) for inputs in X]

    def _best_split(self, X, treatment, y):
        if treatment.size <= 1:
            return None, None
        
        delta_delta_best = 0
        best_thr = None
        best_idx = None
        
        for idx in range(self.n_features_):
            column_values = np.array(X[:, idx])

            unique_values = np.unique(column_values)

            if len(unique_values) > 10:
                percentiles = np.percentile(column_values, [3, 5, 10, 20, 30, 50, 70, 80, 90, 95, 97])
            else:
                percentiles = np.percentile(unique_values, [10, 50, 90])

            threshold_options = np.unique(percentiles)

            for threshold in threshold_options:              
                control = (treatment - 1) * (-1)

                left = np.array([1 if x <= threshold else 0 for x in column_values])
                right = (left - 1) * -1

                left_t = left * treatment
                left_c = left * control

                right_t = right * treatment
                right_c = right * control

                left_t_sum = sum(y * left_t)
                left_c_sum = sum(y * left_c)
                right_t_sum = sum(y * right_t)
                right_c_sum = sum(y * right_c)

                left_t_count = sum(left_t)
                left_c_count = sum(left_c)
                right_t_count = sum(right_t)
                right_c_count = sum(right_c)

                uplift_left = (left_t_sum / left_t_count) - (left_c_sum / left_c_count)
                uplift_right = (right_t_sum / right_t_count) - (right_c_sum / right_c_count)

                delta_delta = np.abs(uplift_left - uplift_right)
                
                if delta_delta > delta_delta_best:
                    if  ((left_t_count + left_c_count) >= self.min_samples_leaf) and \
                        ((right_t_count + right_c_count) >= self.min_samples_leaf) and \
                        (left_t_count >= self.min_samples_leaf_treated) and \
                        (right_t_count >= self.min_samples_leaf_treated) and \
                        (left_c_count >= self.min_samples_leaf_control) and \
                        (right_c_count >= self.min_samples_leaf_control):
                        
                        delta_delta_best = delta_delta
                        best_thr = threshold
                        best_idx = idx
                        
        return best_idx, best_thr

    def _grow_tree(self, X, y, treatment, depth=0):
        control = np.array([1 if t == 0 else 0 for t in treatment])
        predicted_class = sum(y * treatment)/sum(treatment) - sum(y * control)/sum(control)
        node = Node(predicted_class=predicted_class)
        if depth < self.max_depth:
            idx, thr = self._best_split(X, treatment, y)
            if idx is not None:
                indices_left = X[:, idx] < thr
                X_left, t_left, y_left = X[indices_left], treatment[indices_left], y[indices_left]
                X_right, t_right, y_right = X[~indices_left], treatment[~indices_left], y[~indices_left]
                node.feature_index = idx
                node.threshold = thr
                node.left = self._grow_tree(X_left, y_left, t_left, depth + 1)
                node.right = self._grow_tree(X_right, y_right, t_right, depth + 1)
        return node

    def _predict(self, inputs):
        node = self.tree_
        while node.left:
            if inputs[node.feature_index] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.predicted_class

In [24]:
model_params = {
    'max_depth': 3,
    'min_samples_leaf': 6000,
    'min_samples_leaf_treated': 2500,
    'min_samples_leaf_control': 2500
}

In [25]:
model = UpliftTreeRegressor(
    max_depth = 3,
    min_samples_leaf = 6000,
    min_samples_leaf_treated = 2500,
    min_samples_leaf_control = 2500
)

In [26]:
start_time = time.time()
model.fit(X, treatment, y)
print("--- %s seconds ---" % (time.time() - start_time))

--- 4.76221489906311 seconds ---


In [27]:
predicted = model.predict(X)

In [28]:
example_preds = np.load('example_preds_new.npy')

In [29]:
sum(predicted == example_preds)

50000

In [30]:
print(model.tree_.threshold)
print(model.tree_.left.threshold)
print(model.tree_.left.right.threshold)

0.8428329389786856
-0.9878097589516122
0.8401218986161384


### Debug test_3

In [31]:
import pickle
with open('test_3.pkl', 'rb') as ifile:
    test_3 = pickle.load(ifile)

In [32]:
X = test_3['X']
treatment = test_3['treatment']
y = test_3['y']

X_test = test_3['X_test']
pred_right = test_3['pred_right']

model = UpliftTreeRegressor(
    max_depth = 4,
    min_samples_leaf = 1000,
    min_samples_leaf_treated = 400,
    min_samples_leaf_control = 400
)

In [33]:
model.fit(X, treatment, y)

In [34]:
preds = model.predict(X_test)

In [35]:
set(preds)

{-1.176776584810562,
 -0.20841180375754276,
 0.7477398752496119,
 1.9453517933449174,
 2.165655294969757,
 2.739893609566705}

In [36]:
set(pred_right)

{-1.176776584810562,
 -0.20841180375754276,
 0.7477398752496119,
 1.9453517933449174,
 2.165655294969757,
 2.739893609566705}